In [1]:
TEST_A = "/Users/joregan/Playing/rd_ctm_edit/H5C120171011va"

In [2]:
lines = []
with open(TEST_A) as f:
    for line in f.readlines():
        lines.append(line.strip())

In [4]:
def accept_all(lines):
    outlines = []
    for line in lines:
        parts = line.split(" ")
        if parts[-1] == "cor":
            outlines.append(line)
        elif parts[-1] == "sub":
            parts[4] = parts[6]
            parts[-1] = "cor"
            outlines.append(" ".join(parts))
    return outlines

In [ ]:
accept_all(lines)

In [12]:
def ctm_to_timed(lines):
    output = []
    for line in lines:
        parts = line.split(" ")
        start = float(parts[2])
        dur = float(parts[3])
        output.append({
            "start": start,
            "end": start + dur,
            "text": parts[6]
        })
    return output

In [15]:
side_a = ctm_to_timed(accept_all(lines))

In [16]:
phonfile = "/Users/joregan/Playing/rd_phonetic/2442205210012872721_480p.json"

In [17]:
import json
with open(phonfile) as f:
    pieces = json.load(f)

In [21]:
def hf_json_to_timed(data):
    output = []
    for chunk in data["chunks"]:
        output.append({
            "start": chunk["timestamp"][0],
            "end": chunk["timestamp"][1],
            "text": chunk["text"]
        })
    return output

In [23]:
side_b = hf_json_to_timed(pieces)

In [27]:
def prune_to_other(left, right, fudge=0.5):
    safe = left
    working = right
    # if right[0]["start"] > (left[0]["start"] - fudge) and right[-1]["end"] < (left[-1]["end"] + fudge):
    #     safe = right
    #     working = left
    output = []
    for item in working:
        if item["start"] < safe[0]["start"] - fudge:
            continue
        elif item["end"] > safe[-1]["end"] + fudge:
            continue
        else:
            output.append(item)

    return safe, output

In [28]:
new_a, new_b = prune_to_other(side_a, side_b)

In [34]:
new_b[0]

{'start': 27.86, 'end': 27.9, 'text': 'ɑː'}

In [35]:
new_a[0]

{'start': 27.86, 'end': 27.919999999999998, 'text': 'Herr'}

In [36]:
def cost(a, b):
    starts = abs(a["start"] - b["start"])
    ends = abs(a["end"] - b["end"])
    return starts + ends

In [41]:
def in_range(a, b, range=0.2):
    r_start = abs(a["start"] - b["start"]) <= range
    r_end = abs(a["end"] - b["end"]) <= range
    return r_start or r_end

In [42]:
in_range(new_a[0], new_b[-1])

False

In [43]:
in_range(new_a[0], new_b[0])

True

In [30]:
import numpy as np

In [33]:
dist_matrix = np.zeros((len(new_a) + 1, len(new_b) + 1))

In [44]:
for i in range(1, len(new_a) + 1):
    for j in range(1, len(new_b) + 1):
        if not in_range(new_a[i-1], new_b[j-1]):
            continue
        pair_cost = cost(new_a[i-1], new_b[j-1])
        dist_matrix[i, j] = pair_cost

In [ ]:
dist_matrix